# 네이버 플레이스 리뷰 데이터 전처리 

간단하게 형태소 분석기를 사용하여 네이버 플레이스에 입점해있는 카페들의 리뷰 데이터를 전처리하는 과정을 담아보았습니다. Proof of Concept 느낌으로 가볍게 봐주세요~

## 리뷰 데이터 불러오기

In [1]:
import pandas as pd

In [2]:
cafe_reviews = pd.read_csv('data/cafe_reviews.csv')
cafe_reviews.head()

,store,rating,description,reviews
0,오드랑베이커리,4.818182,NaN,"['마농바게트 맛있어요~~', '맛있어요.', '까르보나라 빵 진짜맛있어요', '마..."
1,Bake things better,4.875000,안녕하세요! 베이크 띵스 베러입니다.\n우리 매장은 테이크 아웃을 전문으로 한 베이...,['쿠키만 사려 했는데 스콘이 너무 맛있어보여서… 대체적으로 다 맛있었어요~~ 말차...
2,파파무베이커리,4.656863,아빠가 딸을위해 만드는 건강한빵으로 아이가 아빠를부르는 옹알이의 파파무입니다^^천연...,"['사장님도 친절하고 빵도 맛있어요!', '빵도 케익도 커피도 맛있고 사장님 인성..."
3,텍스처베이커리,4.611111,텍스처 베이커리는 가정식 디저트를 추구합니다.\n꾸며지지 않은 러프한 디저트를 즐겨...,"['키쉬종류가 많아서 좋아요', '매장인테리어 귀엽구 빵도 일반적이지 않은 빵들이 ..."
4,가야베이커리,5.000000,NaN,['규모는 작지만 빵이 맛있고 가격도 저렴해서 자주 방문하고 있습니다. 사장님 친절...


### 체크포인트

- 가게 설명글을 써놓지 않은 가게의 `desciption`에는 NaN 으로 되어있음

- `reviews`의 값이 리스트 같아 보이지만 하나의 문자열로 되어있음

In [3]:
cafe_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   store        934 non-null    object 
 1   rating       865 non-null    float64
 2   description  446 non-null    object 
 3   reviews      934 non-null    object 
dtypes: float64(1), object(3)
memory usage: 29.3+ KB


## description 구두점 제거

가게 설명글에 해당하는 `description` 행의 값들에 구두점을 제거

In [4]:
import string
import ast

In [5]:
def remove_punctuation(row: list) -> list:
    reviews = [x.translate(punctuation_mapping) for x in row]
    return reviews

In [6]:
punctuation_mapping = str.maketrans('', '', string.punctuation)

In [7]:
cafe_reviews['description'] = cafe_reviews['description'].fillna('')
cafe_desc = remove_punctuation(cafe_reviews['description'])
cafe_reviews['description'] = pd.Series(cafe_desc)

## reviews에 각 리뷰마다 구두점 제거

리스트 객체의 메소드를 사용하기 위해 우선 리스트 처럼 보이는 문자열을 리스트로 변환시켜주어야 함

In [8]:
reviews = [ast.literal_eval(x) for x in cafe_reviews['reviews']]
cafe_reviews['reviews'] = pd.Series(reviews)
cafe_reviews['reviews'] = cafe_reviews['reviews'].apply(remove_punctuation)

## 명사만 남기기

형태소 분석기로는 [Okt](https://github.com/open-korean-text/open-korean-text)를 사용하였음

In [9]:
from konlpy.tag import Okt
from tqdm import tqdm

In [10]:
okt = Okt()

`reviews`에 각 리뷰마다 명사를 추출

In [11]:
reviews_in_noun = []
for review in tqdm(cafe_reviews['reviews']):
    store_review = []
    for item in review:
        noun = okt.nouns(item)  # 명사 추출
        noun = ' '.join(noun)  # 띄어쓰기 제거
        store_review.append(noun)
    store_review = list(filter(None, store_review))  # 빈 문자열 제거
    reviews_in_noun.append(store_review)
cafe_reviews['reviews_in_noun'] = pd.Series(reviews_in_noun)

100%|██████████████████████████████████████████████████| 934/934 [00:42<00:00, 22.14it/s]


## 결과물

In [12]:
cafe_reviews

,store,rating,description,reviews,reviews_in_noun
0,오드랑베이커리,4.818182,,"[마농바게트 맛있어요, 맛있어요, 까르보나라 빵 진짜맛있어요, 마농은 필수입니다, ...","[마농 바게트, 까르보나라 빵 진짜, 마농 필수, 빵, 빵 달, 마늘 바게트 안 것..."
1,Bake things better,4.875000,안녕하세요 베이크 띵스 베러입니다\n우리 매장은 테이크 아웃을 전문으로 한 베이커리...,[쿠키만 사려 했는데 스콘이 너무 맛있어보여서… 대체적으로 다 맛있었어요 말차초코는...,"[쿠키 사려 스콘 보 체적 말차 초코 안 크림 치즈 꽤 습, 쿠키 진심 하나 몇개 ..."
2,파파무베이커리,4.656863,아빠가 딸을위해 만드는 건강한빵으로 아이가 아빠를부르는 옹알이의 파파무입니다천연발효...,"[사장님도 친절하고 빵도 맛있어요, 빵도 케익도 커피도 맛있고 사장님 인성이 너무...","[사장 빵, 빵 케익 커피 사장 인성 번창, 최애 동네 빵집, 브랜드 빵 종류 동네..."
3,텍스처베이커리,4.611111,텍스처 베이커리는 가정식 디저트를 추구합니다\n꾸며지지 않은 러프한 디저트를 즐겨보세요,"[키쉬종류가 많아서 좋아요, 매장인테리어 귀엽구 빵도 일반적이지 않은 빵들이 많아서...","[키쉬 종류, 매장 인테리어 빵 일반 빵, 리뷰 보고 곳 당근 케익 맛, 빵 정말 ..."
4,가야베이커리,5.000000,,[규모는 작지만 빵이 맛있고 가격도 저렴해서 자주 방문하고 있습니다 사장님 친절하시...,"[규모 작 빵 가격 자주 방문 사장, 색롤 구입 보기, 옛날 빵 나이 빵집 동네 빵..."
...,...,...,...,...,...
929,평화로빵,4.388889,,"[맛집, 쑥빵 맛있어요, 평범한 동네 빵집이었어요, , 굿, 굿, 좋습니다아으, 굳...","[맛집, 쑥빵, 동네 빵집, 굿, 굿]"
930,한라드블랑,4.625000,제주 프리미엄 케이크 전문점 예약제로 운영되고있습니다 \n최소 23일전까지 카카오...,[사장님 너무너무 친절하시규 포크도 챙겨주셔서 감동ㅠㅠ 크레이크 케이크는 첨인데 쫀...,"[사장 규 포크 감동 레이크 케이크 첨 쫀득하 맛, 진짜 생크림 맛 다음 홀 케이크..."
931,함덕스마일찹쌀꽈배기,4.552632,정성스런 음식\n친절한 서비스\n맛보장\n,"[가게는 아담하지만 핫도그 정말 맛있어요, 맛나요, 평범하지만 맛있어요 기름지지않고...","[가게 핫도그 정말, 맛 요, 쫄깃 가끔 그 시장 찹쌀 꽈배기 맛 설탕 잔뜩, 쫄깃..."
932,해돋이까페,NaN,,[],[]
